In [12]:
!pip install langchain-openai



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [13]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [14]:
load_dotenv()
model = ChatOpenAI()

In [15]:
class BlogState(TypedDict):
    title: str
    outline:str
    content:str
    

In [16]:
def create_outline(state: BlogState) -> BlogState:
    
    title = state['title']
    
    
    prompt = f"generate the detail outline blog on {title}"
    
    outline = model.invoke(prompt).content

    state['outline'] = outline
    
    return state 
    
    

In [19]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    
    prompt = f"write the detailed blog on {title} based on the following outline:\n\n{outline}"
    
    content = model.invoke(prompt).content
    
    state['content'] = content
    
    return state
    
    

In [21]:
graph = StateGraph(BlogState)

#add nodes
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog) 

# add edges

graph.add_edge(START , 'create_outline')
graph.add_edge('create_outline' , "create_blog")
graph.add_edge('create_blog' , END)

workflow = graph.compile()

In [23]:
initial_state = {'title' : 'rise of AI in India'}

final_state = workflow.invoke(initial_state)
print(final_state)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}